In [1]:
import string
import numpy as np

# Decode the two ciphertexts from the Instructors Box below, 
# or the C1, C2 variables - which are the same
#
# We highly recommend that you run your decoding code in the 
# programming language of your choice outside of 
# this environment, as this system does not provide enough 
# computational resources to successfully decode
#
# After decoding the two ciphertexts, 
# replace the plaintext1 and plaintext2 variables below
# with the decoded ciphertexts

# C1 and C2 are messages in english, 
# encoded using string_to_bits, with 7bit ASCII
# and then XOR'd with a secret key
#
# In pseudo-code:
# C1 = XOR(string_to_bits(plaintext1), secret_key)
# C2 = XOR(string_to_bits(plaintext2), secret_key)
C1 = "1010110010011110011111101110011001101100111010001111011101101011101000110010011000000101001110111010010111100100111101001010000011000001010001001001010000000010101001000011100100010011011011011011010111010011000101010111111110010011010111001001010101110001111101010000001011110100000000010010111001111010110000001101010010110101100010011111111011101101001011111001101111101111000100100001000111101111011011001011110011000100011111100001000101111000011101110101110010010100010111101111110011011011001101110111011101100110010100010001100011001010100110001000111100011011001000010101100001110011000000001110001011101111010100101110101000100100010111011000001111001110000011111111111110010111111000011011001010010011100011100001011001101110110001011101011101111110100001111011011000110001011111111101110110101101101001011110110010111101000111011001111"

C2 = "1011110110100110000001101000010111001000110010000110110001101001111101010000101000110100111010000010011001100100111001101010001001010001000011011001010100001100111011010011111100100101000001001001011001110010010100101011111010001110010010101111110001100010100001110000110001111111001000100001001010100011100100001101010101111000100001111101111110111001000101111111101011001010000100100000001011001001010000101001110101110100001111100001011101100100011000110111110001000100010111110110111010010010011101011111111001011011001010010110100100011001010110110001001000100011011001110111010010010010110100110100000111100001111101111010011000100100110011111011001010101000100000011111010010110111001100011100001111100100110010010001111010111011110110001000111101010110101001110111001110111010011111111010100111000100111001011000111101111101100111011001111"
"""
plaintext1 = "Anyone who considers arithmetical methods of producing random digits is, of course, in a state of sin. (John von Neumann)"
plaintext2 = "I visualize a time when we will be to robots what dogs are to humans, and I'm rooting for the machines.  (Claude Shannon)"
"""

'\nplaintext1 = "Anyone who considers arithmetical methods of producing random digits is, of course, in a state of sin. (John von Neumann)"\nplaintext2 = "I visualize a time when we will be to robots what dogs are to humans, and I\'m rooting for the machines.  (Claude Shannon)"\n'

In [2]:
import re, string, copy # os,
from functools import reduce


BITS = ('0', '1')
ASCII_BITS = 7

"""
These functions were provided by Udacity but not used here.
# Below is some code that might be useful
#

def display_bits(b):
    converts list of {0, 1}* to string
    return ''.join([BITS[e] for e in b])  # converts a list of bits to a string of bits 0 and 1

def seq_to_bits(seq):
    return [0 if b == '0' else 1 for b in seq]

def pad_bits(bits, pad):
    pads seq with leading 0s up to length pad
    assert len(bits) <= pad
    return [0] * (pad - len(bits)) + bits

def convert_to_bits(n, pad):  # converts an integer `n` to bit array
    result = []  # an array of bits
    if n == 0:
        return [0]
    while n > 0:
        if n % 2 == 0:
            result = [0] + result
        else:
            result = [1] + result
        n = n // 2
    while len(result) < pad:
        result = [0] + result
    return result    # so print(convert_to_bits(7,7)) will return [0, 0, 0, 0, 1, 1, 1]

def string_to_bits(s):    # so print(string_to_bits('AB')) returns ...
    result = []           # [1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0]
    for c in s:
        result = result + convert_to_bits(ord(c), ASCII_BITS)
    return result

def dec_to_bin(x):    # so print(dec_to_bin(10)) returns 1010
    return int(bin(x)[2:])

def bits_to_char(b):
    assert len(b) == ASCII_BITS
    value = 0
    for e in b:
        value = (value * 2) + e
    return chr(value)

def bits_to_string(b):
    return ''.join([bits_to_char(b[i:i + ASCII_BITS]) 
                    for i in range(0, len(b), ASCII_BITS)])

def list_to_string(p):
    return ''.join(p)

"""
#  ***** The following code was written by Jerald Cogswell to meet the Udacity Lesson 2 Cryptography Challenge. *****
    
msg_sandbox = [['~'] * int(len(C1)/ASCII_BITS), ['_'] * int(len(C2)/ASCII_BITS)] # display list for guess results
msg_undo = [['~'] * int(len(C1)/ASCII_BITS), ['_'] * int(len(C2)/ASCII_BITS)] # display list for guess results

def string_to_ascii(s):
    return [ord(c) for c in s]

def group_bits(s): # Splits a list into 7-bit ascii values
    n = ASCII_BITS
    return list(s[i * n:i * n+n] for i,v in enumerate(s[::n])) #"this is a line split into n characters" 

def a_printable_string(list1):
    result = True
    printable=[x for x in list1 if 31 < x < 123] # eliminate unprintable characters ...
    good_char=[e for e in printable if chr(e) not in ('`','<','>','=','#','/','&','+','\',','_','*','$',';')] # and these, too.
    #          Also see https://www.programiz.com/python-programming/methods/string/strip
    return len(list1) == len(good_char) # True if it's the same list as input

def toggle1_2(v):   # Toggles values between 1 and 2.
    if 0 < v < 3:   # Check for valid input
        return 3-v  # 1 =>2, 2 =>1, else, None

def print_sandbox(): # Display decoded results of guess so far
    print (''.join(msg_sandbox[0])) # print the virgin sandbox for decoded message strings
    print (''.join(msg_sandbox[1]))
    return None
   
common_words = {" that "," have ","would"," there "," their "," about "," people"," think"," these"," because "," intell",
                " with ", " have", " this", " will ", " your ", " from ", " they ", " know ", " want ", " been ", " good", 
                " much ", " some", " time "}

assert len(C1) == len(C2)
"""
User inputs a guess at a word in Message 1 or 2. XOR guess with the message and see what the key produces in
the other message. If the text in the other message looks like a word or word fragment, populate a trial string 
for each message with the guess and the generated text. For short words like "the" it is helpful to add spaces 
like this: " the " to produce more meaningful output.
"""

exit_return = "c"

while exit_return != "exit":
    if exit_return == "u":
        msg_sandbox = copy.deepcopy(msg_undo)
    else:
        msg_undo =  copy.deepcopy(msg_sandbox)
    print_sandbox()
    other_word = [""]
    position = 0
    search_msg = input("Do you want to look in Message 1 or 2? Enter 1 or 2: ")
    if search_msg not in ("1", "2"): # validate user input
        continue
    msg_no = int(search_msg)    # The message number selected for word search
    alt_msg = toggle1_2(msg_no) # The other message number to apply the key to and see if our choice makes sense.
    guess = input("Guess a word. You may include numerals, spaces, comma, and period. GUESS > ")
    s_len = len(guess)
    print ("Looking for", guess, "in message" , str(msg_no), ". String length is" , str(s_len))
    guess_ascii = string_to_ascii(guess)

    if msg_no == 1:
        line1 = group_bits(C1) # Use when we search message 1 for a guess
        line2 = group_bits(C2)
    else:
        line1 = group_bits(C2) # Use when we search message 2 for a guess
        line2 = group_bits(C1) # We now have a list of 7-bit ascii values

    M1 = [int(n,2) for n in line1] # 2 is the number system base. If 2 is changed to 5, you could convert '101' to return 26 for base 5
    M2 = [int(n,2) for n in line2] # 2 is the number system base. If 2 is changed to 5, you could convert '101' to return 26 for base 5
    
    for g in range(0, int(len(M1)-len(guess_ascii))):
        h = g + len(guess_ascii)
        key = list(map(lambda x,y: x^y, M1[g:h], guess_ascii)) # XOR your guess with encoded string to produce a key
        text_string = list(map(lambda x,y: x^y, M2[g:h], key)) # This is a list of ascii codes
        if a_printable_string(text_string):
            is_word = ''.join(chr(c) for c in text_string) # This is the readable version of text_string
#            is_word = ''.join(map(chr,text_string)) # Another way of doing it
            print ("The text in message", alt_msg, "is'", is_word, "'" , "at position ", g)

    possible_word_at = input("What position value looks like valid text? Enter number (or ENTER to pass >)")
    if possible_word_at != "":
        position = int(possible_word_at)
        if -1 < position < len(msg_sandbox[0])+1: # validate input
            end = len(guess_ascii)
            key = list(map(lambda x,y: x^y, M1[position:position+end], guess_ascii))
            other_word_ascii = list(map(lambda x,y: x^y, M2[position:position+end], key))
            other_word = ''.join(chr(c) for c in other_word_ascii)
            msg_sandbox[msg_no-1][position:position+end] = guess
            msg_sandbox[alt_msg-1][position:position+end] = other_word
    else:
        pass
    
    print_sandbox()
    # The following input gives you the option to undo your recent choice, continue, or exit the program.
    exit_return = input("Type 'exit' to terminate, 'u' to undo, or ENTER to continue >")
    
print ("Program end")


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
_________________________________________________________________________________________________________________________
Do you want to look in Message 1 or 2? Enter 1 or 2: 2
Guess a word. You may include numerals, spaces, comma, and period. GUESS >  the 
Looking for  the  in message 2 . String length is 5
The text in message 1 is' 05sd5 ' at position  5
The text in message 1 is' aoipe ' at position  6
The text in message 1 is' .:oe) ' at position  12
The text in message 1 is' nshl  ' at position  13
The text in message 1 is' 'taer ' at position  14
The text in message 1 is' )t:ah ' at position  16
The text in message 1 is' iwe(2 ' at position  23
The text in message 1 is' -9zx% ' at position  25
The text in message 1 is' 2imja ' at position  27
The text in message 1 is' az-(1 ' at position  31
The text in message 1 is' hicto ' at position  37
The text in message 